Copy and paste:
- scheduleProfile_JOBTITLE.py
- actorFunctions_JOBTITLE.py

into \Cluster_User_Emulation_System-main\src\actionScheduler.

In [1]:
count = 1

# Path to ChatGPT poutput file
file_path = '2907_chatGPT_output1.txt'
job_profile_cred = 'config_SoftwareEngineer.ini' # config.ini

Scripts:
- Convert config file to a dictionary.
- scheduleProfile_JOBTITLE.py
- actorFunctions_JOBTITLE.py

-- Admin

In [2]:
# Import libraries

import pandas as pd
from datetime import date

# -------------------------------------------------------------------------------------------------

# Dictionary: All actions and number of parameters.
# Action: (function name, number of parameters)

dict_actions = { "Ping target": ('pingTarget', 4),
                 "Use browser to open URL": ('openURL', 2),
                 "Open Word document": ('openWord', 2),
                 "Run one command": ('runOneCmd', 5),
                 "Clone repo": ('cloneRepo', 3),
                 "Hold Zoom meeting": ('holdZoom', 2),
                 "Take screenshot of website": ('takeScreenshot', 3),
                 "Send Telegram": ('sendTelegram', 3),
                 "Send email": ('sendEmail', 5) }

-- Script: Convert config file to a dictionary.

In [3]:
def convert_config_to_dictionary(config_file):

    config_dict = {}

    with open(config_file) as f:

        for line in f:

            if '[' in line:
                key = line.strip().replace('[', '').replace(']', '')
                config_dict[key] = {}
                action = key
            elif '=' in line:
                param, value = key, value = line.strip().split('=', 1)
                config_dict[action][param.strip()] = value.strip()
    return config_dict

convert_config_to_dictionary(job_profile_cred)

{'cloneRepo': {'dirPath': 'C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\UtilsFunc'},
 'takeScreenshot': {'outputFolder': 'C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\outputFolder'},
 'sendTelegram': {'token': '6725278465:AAFgXrznCf9vXkHUDGc8aoBJG6qEv3xxQrY',
  'chatID': '448238802'},
 'sendEmail': {'username': 'nus.capstone.test',
  'appPassword': 'exqrbskdajyjmnuj',
  'recipients': 'nus.capstone.test@gmail.com'}}

-- Script:
For ChatGPT output.

In [4]:
def extract_info_from_text(file_path):
    job_title = ""
    job_description = ""
    csv_schedule = []
    reading_schedule = False

    with open(file_path, 'r') as file:
        for line in file:
            # Strip extra whitespace and check if line is part of CSV schedule
            stripped_line = line.strip()
            if stripped_line.startswith("Time,Action"):
                reading_schedule = True
                csv_schedule.append(stripped_line)  # Start capturing the CSV header
                continue
            
            if reading_schedule:
                if stripped_line:
                    csv_schedule.append(stripped_line)
                continue
            
            # Capture job title
            if stripped_line.lower().startswith("job title:"):
                job_title = stripped_line.split(":", 1)[1].strip()
            
            # Capture description
            if stripped_line.lower().startswith("job task:"):
                job_description = stripped_line.split(":", 1)[1].strip()

    return job_title, job_description, "\n".join(csv_schedule)

# Extract information from the text file
job_title, job_description, csv_schedule = extract_info_from_text(file_path)

print("Job Title:", job_title)
print("Job Task:", job_description)
print("CSV Schedule:")
print(csv_schedule)

jobTitle = job_title.replace(" ", "")
jobTitle += '_' + str(date.today()) + '_' + str(count)
if '-' in jobTitle:
    jobTitle = jobTitle.replace("-", "_")

with open('scheduleCSV_' + jobTitle + '.csv', 'w') as f:
    f.write(csv_schedule)

Job Title: Software Engineer
Job Task: <To be filled in>
CSV Schedule:
Time,Action,Description,Parameter1,Value1,Parameter2,Value2,Parameter3,Value3,Parameter4,Value4,Parameter5,Value5
09:00,Use browser to open URL,Check emails,url,"https://mail.example.com",watchInterval,15,,,,
09:15,Use browser to open URL,Update project tracking tools,url,"https://project-tracker.example.com",watchInterval,15,,,,
09:30,Hold Zoom meeting,Attend daily stand-up meeting,username,"user@example.com",meetingURL,"https://zoom.us/j/1234567890",,,,
10:00,Clone repo,Get latest codebase for review,dirPath,"C:\\Projects\\Repo",repoName,"project-repo",repoURL,"https://github.com/user/project-repo",,,
10:15,Run one command,Review and merge pull requests,cmdID,"mergePRs",console,True,cmdStr,"git merge origin/main",repeat,1,interval,0.5
10:45,Clone repo,Start working on new feature,dirPath,"C:\\Projects\\Repo",repoName,"feature-branch",repoURL,"https://github.com/user/project-repo",,,
11:00,Run one command,Build and

In [5]:
jobTitle

'SoftwareEngineer_2024_07_29_1'

-- Script: scheduleProfile_JOBTITLE.py

In [6]:
# Admin:
# - Define ChatGPT's schedule CSV file path and job title.
# - Read CSV file into a DataFrame.

scheduleCSV_filePath = 'scheduleCSV_' + jobTitle + '.csv'

df_schedule = pd.read_csv(scheduleCSV_filePath)

In [7]:
# Template: scheduleProfile_JOBTITLE.py - Header

actorName_IP = 'Char[192.168.57.10]'

scheduleProfile_header = f"""import datetime
import actorFunctions{jobTitle} as actorFunctions

ACTOR_NAME = '{actorName_IP}'

dailyTaskList = []
randomTaskList = []
weeklyTaskList = []

"""

In [8]:
# Template: scheduleProfile_JOBTITLE.py - Action functions

scheduleProfile_Actions = ""

for row in df_schedule.itertuples(index=True, name='Pandas'):
    time = row.Time.replace(':', '')

    if row.Action not in dict_actions:
        continue
#         action = f"""
# action_{time} = {{
#     'time': '{row.Time}',
#     'name': '{row.Time}_{row.Action.replace(" ", "_")}',
#     'actionFunc': actorFunctions.func_{time},
#     'parallelTH': False,
#     'actDetail': "{row.Action}",
#     'actDesc': "{row.Description}"
#     }}"""

    else:
        action = f"""
action_{time} = {{
    'time': '{row.Time}',
    'name': '{row.Time}_{dict_actions[row.Action][0]}',
    'actionFunc': actorFunctions.func_{time},
    'parallelTH': False,
    'actDetail': "{row.Action}",
    'actDesc': "{row.Description}"
    }}

dailyTaskList.append(action_{time})"""

    scheduleProfile_Actions += action + "\n"
    

In [9]:
jobTitle

'SoftwareEngineer_2024_07_29_1'

In [10]:
# Script: Create scheduleProfile_JOBTITLE.py

action_banner = "# -------------------------------------\n# Actions\n# -------------------------------------\n"

scheduleProfile_complete = scheduleProfile_header + action_banner + scheduleProfile_Actions

with open('scheduleProfile_' + jobTitle + '.py', 'w') as f:
    f.write(scheduleProfile_complete)

-- Script: actorFunctionsJOBTITLE.py

In [11]:
# df_schedule

In [12]:
actorFunctions_header = f"""# Import libraries
import char_fiveActions

PORT = 443 # Port to download the server certificate most server use 443.

"""

In [13]:
# Template: actorFunctionsJOBTITLE.py - Action configurations

# actorFunctions_ActionsConfigs = ""

# for row in df_schedule.itertuples(index=True, name='Pandas'):
    
#     if row.Action not in dict_actions:
#         continue

#     action_config_name, num_params = dict_actions[row.Action]
#     time = row.Time.replace(":", "")
    
#     # Start building the action config string
#     config = f"\n{action_config_name}Config_{time} = {{\n"
    
#     # Loop through the number of parameters expected for this action
#     for i in range(1, num_params + 1):
#         param = getattr(row, f'Parameter{i}', None)
#         value = getattr(row, f'Value{i}', None)
        
#         if param is not None and value is not None:  # Ensure the parameter and value are not None
#             # Attempt to keep numbers as floats or convert them to string if necessary
#             try:
#                 # Convert value directly to float
#                 converted_value = float(value)
#                 config += f'    "{param}": {converted_value},\n'
#             except ValueError:
#                 # If conversion fails, treat it as a string
#                 config += f'    "{param}": "{value}",\n'
    
#     config += "}\n"
#     actorFunctions_ActionsConfigs += config


    # Template: actorFunctionsJOBTITLE.py - Action configurations

# -------------------------------------------------------------------------------------------------
# New actorFunctions_ActionsConfigs
# -------------------------------------------------------------------------------------------------

localDetails_SoftwareEngineer = convert_config_to_dictionary(job_profile_cred)

def auto_convert_type(param, value):
    try:
        # Convert value directly to float
        converted_value = float(value)
        return f'    "{param}": {converted_value},\n'
    except ValueError:
        # If conversion fails, treat it as a string
        escaped_value = value.replace('\\', '\\\\')
        return f'    "{param}": "{escaped_value}",\n'


actorFunctions_ActionsConfigs2 = ""

for row in df_schedule.itertuples(index=True, name='Pandas'):
    
    # Skip actions not in the dictionary
    if row.Action not in dict_actions:
        continue

    # row.Action: Use browser to open URL
    # dict_actions[row.Action]: ('openURL', 2)
    action_config_name, num_params = dict_actions[row.Action] # action_config_name: openURL, num_params: 2
    time = row.Time.replace(":", "")
    
    # Start building the action config string
    config = f"\n{action_config_name}Config_{time} = {{\n"
    
    # Loop through the number of parameters expected for this action
    for i in range(1, num_params + 1):
        # param: URL, value: https://www.google.com
        param = getattr(row, f'Parameter{i}', None)
        value = getattr(row, f'Value{i}', None)

        if dict_actions[row.Action][0] in localDetails_SoftwareEngineer and param in localDetails_SoftwareEngineer[dict_actions[row.Action][0]]:
            config += auto_convert_type(param, localDetails_SoftwareEngineer[dict_actions[row.Action][0]][param])
        
        else:
            config += auto_convert_type(param, value)
    
    config += "}\n"
    actorFunctions_ActionsConfigs2 += config

print(actorFunctions_ActionsConfigs2)


openURLConfig_0900 = {
    "url": "https://mail.example.com",
    "watchInterval": 15.0,
}

openURLConfig_0915 = {
    "url": "https://project-tracker.example.com",
    "watchInterval": 15.0,
}

holdZoomConfig_0930 = {
    "username": "user@example.com",
    "meetingURL": "https://zoom.us/j/1234567890",
}

cloneRepoConfig_1000 = {
    "dirPath": "C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\UtilsFunc",
    "repoName": "project-repo",
    "repoURL": "https://github.com/user/project-repo",
}

runOneCmdConfig_1015 = {
    "cmdID": "mergePRs",
    "console": "True",
    "cmdStr": "git merge origin/main",
    "repeat": 1.0,
    "interval": 0.5,
}

cloneRepoConfig_1045 = {
    "dirPath": "C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\UtilsFunc",
    "repoName": "feature-branch",
    "repoURL": "https://github.com/user/project-repo",
}

runOneCmdConfig_1100 = {
    "

In [14]:
actorFunctions_functions = ""

for row in df_schedule.itertuples(index=True, name='Pandas'):

    if row.Action not in dict_actions:
        continue

    time = row.Time.replace(':', '')
    action_function = f"""def func_{time}():
    # {row.Action} - {row.Description}
    
    # Insert function
    actor = char_fiveActions.{dict_actions[row.Action][0]}Actor()
    actor.{dict_actions[row.Action][0]}({dict_actions[row.Action][0]}Config_{time})
    
#-----------------------------------------------------------------------------

"""
    actorFunctions_functions += action_function


In [15]:
config_banner = "# -------------------------------------\n# Configuration for each action\n# -------------------------------------\n"

actorFunctions_complete = actorFunctions_header + config_banner + actorFunctions_ActionsConfigs2 + action_banner + actorFunctions_functions

with open('actorFunctions' + jobTitle + '.py', 'w') as f:
    f.write(actorFunctions_complete)

-- Script: Add in user's information to config.

In [16]:
# Template: actorFunctionsJOBTITLE.py - Action configurations

actorFunctions_ActionsConfigs2 = ""

for row in df_schedule.itertuples(index=True, name='Pandas'):
    
    # Skip actions not in the dictionary
    if row.Action not in dict_actions:
        continue

    # row.Action: Use browser to open URL
    # dict_actions[row.Action]: ('openURL', 2)
    action_config_name, num_params = dict_actions[row.Action] # action_config_name: openURL, num_params: 2
    time = row.Time.replace(":", "")
    
    # Start building the action config string
    config = f"\n{action_config_name}Config_{time} = {{\n"
    
    # Loop through the number of parameters expected for this action
    for i in range(1, num_params + 1):
        # param: URL, value: https://www.google.com
        param = getattr(row, f'Parameter{i}', None)
        value = getattr(row, f'Value{i}', None)

        if dict_actions[row.Action][0] in localDetails_SoftwareEngineer and param in localDetails_SoftwareEngineer[dict_actions[row.Action][0]]:
            config += auto_convert_type(param, localDetails_SoftwareEngineer[dict_actions[row.Action][0]][param])
        
        else:
            config += auto_convert_type(param, value)
    
    config += "}\n"
    actorFunctions_ActionsConfigs2 += config

print(actorFunctions_ActionsConfigs2)


openURLConfig_0900 = {
    "url": "https://mail.example.com",
    "watchInterval": 15.0,
}

openURLConfig_0915 = {
    "url": "https://project-tracker.example.com",
    "watchInterval": 15.0,
}

holdZoomConfig_0930 = {
    "username": "user@example.com",
    "meetingURL": "https://zoom.us/j/1234567890",
}

cloneRepoConfig_1000 = {
    "dirPath": "C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\UtilsFunc",
    "repoName": "project-repo",
    "repoURL": "https://github.com/user/project-repo",
}

runOneCmdConfig_1015 = {
    "cmdID": "mergePRs",
    "console": "True",
    "cmdStr": "git merge origin/main",
    "repeat": 1.0,
    "interval": 0.5,
}

cloneRepoConfig_1045 = {
    "dirPath": "C:\\Users\\leech\\Downloads\\00. School\\Capstone\\Cluster_User_Emulation_System-main\\src\\actionScheduler\\UtilsFunc",
    "repoName": "feature-branch",
    "repoURL": "https://github.com/user/project-repo",
}

runOneCmdConfig_1100 = {
    "